In [3]:
!pip install opencv-python==4.6.0.66
!pip install mediapipe==0.10.9
!pip install numpy==1.23
!pip install playsound==1.2.2

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math
import threading
import sys
import os

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

p_olho_esq = [385, 380, 387, 373, 362, 263]
p_olho_dir = [160, 144, 158, 153, 33, 133]
p_olhos = p_olho_esq + p_olho_dir

def calculo_ear(face, p_olho_dir, p_olho_esq):
    try:
        face = np.array([[coord.x, coord.y] for coord in face])
        face_esq = face[p_olho_esq,:]
        face_dir = face[p_olho_dir,:]

        ear_esq = (np.linalg.norm(face_esq[0]-face_esq[1]) + np.linalg.norm(face_esq[2]-face_esq[3])) / (2 * (np.linalg.norm(face_esq[4]-face_esq[5])))
        ear_dir = (np.linalg.norm(face_dir[0]-face_dir[1]) + np.linalg.norm(face_dir[2]-face_dir[3])) / (2 * (np.linalg.norm(face_dir[4]-face_dir[5])))
    except:
        return 0.0
    return (ear_esq + ear_dir) / 2

p_boca = [82, 87, 13, 14, 312, 317, 78, 308]

def calculo_mar(face, p_boca):
    try:
        face = np.array([[coord.x, coord.y] for coord in face])
        face_boca = face[p_boca,:]

        mar = (np.linalg.norm(face_boca[0]-face_boca[1]) +
               np.linalg.norm(face_boca[2]-face_boca[3]) +
               np.linalg.norm(face_boca[4]-face_boca[5])) / (2*(np.linalg.norm(face_boca[6]-face_boca[7])))
    except:
        return 0.0
    return mar

ear_limiar = 0.3
mar_limiar = 0.2

dormindo = 0
contagem_piscadas = 0
c_tempo = 0
contagem_temporaria = 0
contagem_lista = [0]
t_inicial = 0
t_piscadas = time.time()

# Contagem de bocejo
tempo_boca_aberta = 0
bocejos = 0
BOCEJO_TEMPO = 3.0
mostrar_alerta_bocejo = False
tempo_alerta_bocejo = 0

# DETECÇÃO HOLISTIC (Pose + Mãos)
mp_holistic = mp.solutions.holistic

hand_conn_spec = mp_drawing.DrawingSpec(color=(255,255,255), thickness=2)
hand_lm_spec   = mp_drawing.DrawingSpec(color=(0,0,255),   thickness=2, circle_radius=3)

pose_conn_spec = mp_drawing.DrawingSpec(color=(255,255,255), thickness=3)
pose_lm_spec   = mp_drawing.DrawingSpec(color=(0,165,255),  thickness=2, circle_radius=3)

# SOM DO ALERTA (não trava)
USE_WINDOWS_BEEP = sys.platform.startswith("win")
ALERTA_SOUND_FILE = "alerta.mp3"
ALERT_INTERVAL = 6.0
_last_alert_time = 0.0

if not USE_WINDOWS_BEEP:
    try:
        from playsound import playsound
        PLAYSOUND_AVAILABLE = True
    except:
        PLAYSOUND_AVAILABLE = False
else:
    PLAYSOUND_AVAILABLE = False

def safe_play_sound():
    global _last_alert_time
    now = time.time()
    if now - _last_alert_time < ALERT_INTERVAL:
        return
    _last_alert_time = now

    if USE_WINDOWS_BEEP:
        import winsound
        winsound.Beep(1000, 400)
    else:
        if PLAYSOUND_AVAILABLE and os.path.exists(ALERTA_SOUND_FILE):
            threading.Thread(target=lambda: playsound(ALERTA_SOUND_FILE), daemon=True).start()
        else:
            print("\a", end="", flush=True)


# CAPTURA DA CÂMERA
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    refine_face_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as holistic:

    while cap.isOpened():
        ok, frame = cap.read()
        if not ok:
            continue

        comprimento, largura, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(rgb)
        frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)

        # DESENHO DE CORPO E MÃOS
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                      landmark_drawing_spec=pose_lm_spec,
                                      connection_drawing_spec=pose_conn_spec)

        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                      landmark_drawing_spec=hand_lm_spec,
                                      connection_drawing_spec=hand_conn_spec)

        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                      landmark_drawing_spec=hand_lm_spec,
                                      connection_drawing_spec=hand_conn_spec)

        # DETECÇÃO FACIAL 
        if results.face_landmarks:
            face_landmarks = results.face_landmarks
            face = face_landmarks.landmark

            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,102,102), thickness=1, circle_radius=1),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(102,204,0), thickness=1))

            # pontos olhos e boca
            for i, lm in enumerate(face):
                if i in p_olhos or i in p_boca:
                    coord = mp_drawing._normalized_to_pixel_coordinates(lm.x, lm.y, largura, comprimento)
                    if coord:
                        cv2.circle(frame, coord, 2, (255,0,0), -1)

            # EAR / MAR
            ear = calculo_ear(face, p_olho_dir, p_olho_esq)
            mar = calculo_mar(face, p_boca)

            cv2.rectangle(frame, (0,1), (290,230), (58,58,55), -1)
            cv2.putText(frame, f"EAR: {round(ear,2)}", (1, 24),
                        cv2.FONT_HERSHEY_DUPLEX, 0.9, (255,255,255), 2)

            cv2.putText(frame, f"MAR: {round(mar,2)} {'Aberta' if mar>=mar_limiar else 'Fechada'}",
                        (1, 50), cv2.FONT_HERSHEY_DUPLEX, 0.9, (255,255,255), 2)
            
        # INCLINAÇÃO DO OMBRO 
        inclinacao = 0
        if results.pose_landmarks:
            l = results.pose_landmarks.landmark
            ombro_esq = l[mp_holistic.PoseLandmark.LEFT_SHOULDER]
            ombro_dir = l[mp_holistic.PoseLandmark.RIGHT_SHOULDER]

            dx = ombro_dir.x - ombro_esq.x
            dy = ombro_dir.y - ombro_esq.y
            inclinacao = math.degrees(math.atan2(dy, dx))

        cv2.putText(frame, f"Inclinacao Ombro:", (1, 185),
                    cv2.FONT_HERSHEY_DUPLEX, 0.6, (255,255,255), 1)
        cv2.putText(frame, f"{round(inclinacao,2)} graus", (1, 205),
                    cv2.FONT_HERSHEY_DUPLEX, 0.7, (255,255,255), 2)

        # BOCEJO 
        if mar >= mar_limiar:
            if tempo_boca_aberta == 0:
                    tempo_boca_aberta = time.time()
            else:
                if time.time() - tempo_boca_aberta >= BOCEJO_TEMPO:
                        bocejos += 1
                        tempo_boca_aberta = 0

                        mostrar_alerta_bocejo = True
                        tempo_alerta_bocejo = time.time()
                        safe_play_sound()
        else:
                tempo_boca_aberta = 0

        cv2.putText(frame, f"Bocejos: {bocejos}", (1, 150),
                    cv2.FONT_HERSHEY_DUPLEX, 0.9, (109,233,219), 2)

        # LÓGICA ORIGINAL DE SONO/PISCADAS
        if ear < ear_limiar and mar < mar_limiar:
            t_inicial = time.time() if dormindo == 0 else t_inicial
            contagem_piscadas += 1 if dormindo == 0 else 0
            dormindo = 1
        if (dormindo == 1 and ear >= ear_limiar) or (ear <= ear_limiar and mar >= mar_limiar):
                dormindo = 0

        t_final = time.time()
        tempo_decorrido = t_final - t_piscadas

        if tempo_decorrido >= (c_tempo + 1):
            c_tempo = tempo_decorrido
            piscadas_ps = contagem_piscadas - contagem_temporaria
            contagem_temporaria = contagem_piscadas
            contagem_lista.append(piscadas_ps)
            contagem_lista = contagem_lista[-60:]
        piscadas_pm = 15 if tempo_decorrido <= 60 else sum(contagem_lista)

        cv2.putText(frame, f"Piscadas: {contagem_piscadas}", (1, 120),
                        cv2.FONT_HERSHEY_DUPLEX, 0.9, (109,233,219), 2)

        tempo_sono = (t_final - t_inicial) if dormindo == 1 else 0.0
        cv2.putText(frame, f"Tempo: {round(tempo_sono,3)}", (1, 80),
                    cv2.FONT_HERSHEY_DUPLEX, 0.9, (255,255,255), 2)

        # ALERTA FINAL (som + caixa)
        if piscadas_pm < 10 or tempo_sono >= 1.5:
            cv2.rectangle(frame, (30, 400), (610, 452), (109,233,219), -1)
            cv2.putText(frame, "Pode ser que voce esteja com sono,", (60, 420),
                        cv2.FONT_HERSHEY_DUPLEX, 0.85, (58,58,55), 1)
            cv2.putText(frame, "considere descansar.", (180, 450),
                        cv2.FONT_HERSHEY_DUPLEX, 0.85, (58,58,55), 1)

            safe_play_sound()
            
        if mostrar_alerta_bocejo:
            if time.time() - tempo_alerta_bocejo <=1.5:
                cv2.rectangle(frame, (30, 400), (610, 452), (109,233,219), -1)
                cv2.putText(frame, "Pode ser que voce esteja fadigado,", (60, 420),
                            cv2.FONT_HERSHEY_DUPLEX, 0.85, (58,58,55), 1)
                cv2.putText(frame, "considere descansar.", (180, 450),
                                cv2.FONT_HERSHEY_DUPLEX, 0.85, (58,58,55), 1)
            else: 
                mostrar_alerta_bocejo = False

            safe_play_sound()

        cv2.imshow("Camera", frame)
        if cv2.waitKey(10) & 0xFF == ord('c'):
            break

cap.release()
cv2.destroyAllWindows()